In [1]:
# Load Packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
import copy
from numpy.linalg import multi_dot
from scipy.stats import norm
import matplotlib.pyplot as plt
import math


#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, prev_weekday, \
                                                     find_wkly_expries,date_of_prev_thurs, \
                                                     generate_daily_dates_each_month

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data

# Import functions of option Greeks
from ipynb.fs.full.user_defined_vik_functions import call_delta_with_spot_and_div, put_delta_with_spot_and_div

#Import implied volatility processing functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_iv, gen_interpolated_iv

#Import Risk-free interest rate generation functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_ir

# Import risk free interest rate function
from ipynb.fs.full.user_defined_vik_functions import get_risk_free_rate_from_exact_date


In [22]:
##########################################
#      Dynamic Hedging by Futures        #
#########################################

#Index of Interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "OTM"

#Product type to hedge: either "CE" or "PE"
# prod_type = "CE"
prod_type = "PE"

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path


# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}
# periods_of_interest = {2019: [10]}
#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]


In [23]:
# Dynamic Hedging iv from vol surface df

def get_iv_linearIntpl(df_iv, iv_date, option_moneyness):
    
    df_date = df_iv[df_iv['Date'] == iv_date]
    df_date = df_date.sort_values(['T', 'Moneyness'], ascending=[False, True]).reset_index()
            
    # np.unique also sorts the array - we need to re-sort in descending order
    # This is required as last week has only one tenor and when filtering, we need to filter as 
    # tenor_list[0]
    # We need vol of  option to be hedged for dynamic hedge - so need minus below
    
    t_list = -np.sort(np.unique(np.array(-df_date['T'])))
    tenor_list = [str(int(round(t* 365))) + "D" for t in t_list]
            
    if (len(tenor_list) > 2):
        print("There are more than 2 tenors in the vol surface - Please check!")

    df_tenor = df_date[df_date['Tenor'] == tenor_list[0]]

    x = np.array(df_tenor["Moneyness"])
    y = np.array(df_tenor["Impl_Vol"])
    
    iv = np.interp(option_moneyness, x, y)
    
    return iv
    


In [24]:

#Dynamic Hedging Function
def dynamic_hedge_pfl_error(mthly_expiries_list, dict_wkly_expiries_each_month, mthly_mkt_data,\
                           prod_type, prod_moneyness, holidays_list, \
                           output_path, stock_ident):
        
    mth_list_of_df = []
    df_strikes_hedged = pd.read_csv(output_path + "B1A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_strikes_hedged.csv")

    for each_month in mthly_expiries_list:
        strike = df_strikes_hedged[df_strikes_hedged['Date'] == each_month.strftime("%d-%b-%Y")]['Strike'].iloc[0]
        df_month = mthly_mkt_data[get_mthly_df_name_from_expiry(each_month, prod_type, holidays_list, stock_ident=stock_ident)]

# Remove the below two lines - if we need to use "Close price" instead of "Settlement Price"

        df_month['Date'] = df_month.apply(lambda x: x['Date'].split("-")[0] + "-" + \
                                  x['Date'].split("-")[1]  + "-"  + "20" + x['Date'].split("-")[2] \
                                  if (len(x['Date'].split("-")[2]) == 2) else x['Date'], axis=1) 
        df_month['Expiry'] = df_month.apply(lambda x: x['Expiry'].split("-")[0] + "-" + \
                                  x['Expiry'].split("-")[1]  + "-"  + "20" + x['Expiry'].split("-")[2] \
                                  if (len(x['Expiry'].split("-")[2]) == 2) else x['Expiry'], axis=1)

        if (prod_type == "CE"):
            df_month['Close'] = df_month.apply(lambda x: max(x['Underlying Value'] - x['Strike Price'], 0) if (x['Date'] == x['Expiry']) else \
                                                 x['Settle Price'], axis=1)
        else:
            df_month['Close'] = df_month.apply(lambda x: max(x['Strike Price']-x['Underlying Value'], 0) if (x['Date'] == x['Expiry']) else \
                                                 x['Settle Price'], axis=1)            
        df_month = df_month[['Date', 'Expiry', 'Underlying Value', 'Strike Price', 'Close']]
        df_month = df_month[df_month['Strike Price'] == strike]
        
        start_day = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][0].strftime("%d-%b-%Y")
        df_fut = mthly_mkt_data['df_FUTIDX_' + stock_ident + '_' + start_day + '_TO_' + each_month.strftime("%d-%b-%Y")]
        df_fut['Date'] = df_fut.apply(lambda x: x['Date'].split("-")[0] + "-" + \
                                  x['Date'].split("-")[1]  + "-"  + "20" + x['Date'].split("-")[2] \
                                  if (len(x['Date'].split("-")[2]) == 2) else x['Date'], axis=1) 
        df_fut['Expiry'] = df_fut.apply(lambda x: x['Expiry'].split("-")[0] + "-" + \
                                  x['Expiry'].split("-")[1]  + "-"  + "20" + x['Expiry'].split("-")[2] \
                                  if (len(x['Expiry'].split("-")[2]) == 2) else x['Expiry'], axis=1)        
        
        # Remove the following lines if required to use close price instead of settlement price
        df_fut['Close'] = df_fut['Settle Price']

        df_month = df_month.merge(df_fut, how ='left', left_on='Date', right_on='Date', suffixes=('', '_FUT'))
#         print(df_fut)
        df_month = df_month[['Date', 'Expiry', 'Underlying Value', 'Strike Price', 'Close', 'Close_FUT']]
        
            
        df_month['Date'] = df_month.apply(lambda x: x['Date'].split("-")[0] + "-" + \
                                                        x['Date'].split("-")[1]  + "-"  + \
                                                        "20" + x['Date'].split("-")[2] \
                                                        if (len(x['Date'].split("-")[2]) == 2) else x['Date'], axis=1)  
        
        df_month['Expiry'] = df_month.apply(lambda x: x['Expiry'].split("-")[0] + "-" + \
                                                        x['Expiry'].split("-")[1]  + "-"  + \
                                                        "20" + x['Expiry'].split("-")[2] \
                                                        if (len(x['Expiry'].split("-")[2]) == 2) else x['Expiry'], axis=1)  


        df_month["Moneyness"] = np.divide(df_month['Underlying Value'], df_month['Strike Price'])

        #Vol Surface Input
        df_full_iv = pd.read_csv(output_path + "A3_" + stock_ident + "_implied_vol_surface.csv")
        df_iv = df_full_iv[df_full_iv['Month_Expiry'] == each_month.strftime("%d-%b-%Y")]
        
        df_month_to_find_delta = copy.deepcopy(df_month) 

        df_month_to_find_delta = df_month_to_find_delta.assign(x=pd.to_datetime(df_month_to_find_delta['Date'])).sort_values('x').drop('x', 1).reset_index(drop=True)
        
        # This is to remove dates such as holidays as this is considered while generating iv
        unique_iv_dates = np.unique(np.array(df_iv.loc[:,'Date'])).reshape(-1)
        
        df_dates_iv_file = pd.DataFrame({"IV_Dates": unique_iv_dates, "Indicator": [1 for i in range(0, unique_iv_dates.shape[0])]})
        df_month_to_find_delta = pd.merge(df_month_to_find_delta, df_dates_iv_file, \
                                          how = 'left', left_on = ['Date'], right_on = ['IV_Dates'])
        df_month_to_find_delta = df_month_to_find_delta[df_month_to_find_delta["Indicator"] == 1]
#         print(df_month_to_find_delta)
        # Smile Vol
        df_month_to_find_delta['iv'] = df_month_to_find_delta.apply(lambda x: get_iv_linearIntpl(df_iv, x['Date'], x['Moneyness']) \
                                                                    if (datetime.strptime(x['Date'], "%d-%b-%Y").date() < each_month) else \
                                                                    0.0, axis=1)     
        df_month_to_find_delta['r_f'] = df_month_to_find_delta.apply(lambda x: get_risk_free_rate_from_exact_date(datetime.strptime(x['Date'], "%d-%b-%Y").date()), axis=1)   
        df_month_to_find_delta['expiry_dt'] = df_month_to_find_delta.apply(lambda x: float((each_month - datetime.strptime(x['Date'], "%d-%b-%Y").date()).days) / 365, axis=1)
        df_month_to_find_delta['r'] = df_month_to_find_delta.apply(lambda x: np.log(x['Close_FUT']/x['Underlying Value'])/x['expiry_dt'], axis=1)
        
        if (prod_type == "CE"):
            df_month_to_find_delta['delta'] = df_month_to_find_delta.apply(lambda x: \
                                              call_delta_with_spot_and_div(x['Underlying Value'], x['Strike Price'], \
                                                                   x['r'], x['r_f'], x['iv'], x['expiry_dt']) \
                                              if (datetime.strptime(x['Date'], "%d-%b-%Y").date() < each_month) else \
                                              x['Underlying Value'] * 0.0, axis=1)
        else:
            df_month_to_find_delta['delta'] = df_month_to_find_delta.apply(lambda x: \
                                              put_delta_with_spot_and_div(x['Underlying Value'], x['Strike Price'], \
                                                                   x['r'], x['r_f'], x['iv'], x['expiry_dt']) \
                                              if (datetime.strptime(x['Date'], "%d-%b-%Y").date() < each_month) else \
                                              x['Underlying Value'] * 0.0, axis=1)
        
        
        #Lag Variables
        df_month_to_find_delta['delta_lag'] = [0.0] + list(df_month_to_find_delta['delta'])[:-1]
        df_month_to_find_delta['underlying_lag'] = [0.0] + list(df_month_to_find_delta['Underlying Value'])[:-1]
        df_month_to_find_delta['Close_lag'] = [0.0] + list(df_month_to_find_delta['Close'])[:-1]
        df_month_to_find_delta['Date_lag'] = [start_day] + list(df_month_to_find_delta['Date'])[:-1]
        
        df_month_to_find_delta.loc[0, 'Close_lag'] = df_month_to_find_delta.loc[0, 'Close'] 
        df_month_to_find_delta.loc[0, 'underlying_lag'] = df_month_to_find_delta.loc[0, 'Underlying Value'] 
        df_month_to_find_delta.loc[0, 'Date_lag'] = df_month_to_find_delta.loc[0, 'Date'] 
        df_month_to_find_delta.loc[0, 'delta_lag'] = df_month_to_find_delta.loc[0, 'delta']
        
        df_month_to_find_delta['delta_lag'] = [0.0] + list(df_month_to_find_delta['delta'])[:-1]

        df_month_to_find_delta['hedge_dt'] = df_month_to_find_delta.apply(lambda x: float((datetime.strptime(x['Date'], "%d-%b-%Y").date() - \
                                                                                           datetime.strptime(x['Date_lag'], "%d-%b-%Y").date()).days) / 365, axis=1)
        
        
# We find the amount of stocks purchased to form the dynamic hedge pfl
# We need to find teh stocks bought as of previous to build dynamic pfl yesterday
        df_month_to_find_delta['new_stocks_purchase_qty'] = df_month_to_find_delta['delta'] - df_month_to_find_delta['delta_lag']
        df_month_to_find_delta['new_stocks_purchase_qty_lag'] = [0.0] + list(df_month_to_find_delta['new_stocks_purchase_qty'])[:-1]
        df_month_to_find_delta['abs_new_stocks_purchase_qty_lag'] = df_month_to_find_delta['new_stocks_purchase_qty_lag'].abs()
        equity_trans_cost = 0.0000345

#         df_month_hedge_error = df_month_to_find_delta[df_month_to_find_delta['Date'] != df_month_to_find_delta['Date'].iloc[0]]
        df_month_hedge_error = df_month_to_find_delta
        df_month_hedge_error['Hedge Error Date'] = df_month_hedge_error.loc[:, 'Date'] 
        
        df_month_hedge_error['Dynamic Hedge Pfl Value'] = df_month_to_find_delta.apply(lambda x: (x['Close_lag'] - x['abs_new_stocks_purchase_qty_lag'] * equity_trans_cost - \
                                                                                                    x['delta_lag'] * x['underlying_lag']) \
                                                                                                    * np.exp(x['r_f'] * x['hedge_dt']) + \
                                                                                                    x['delta_lag'] * x['Underlying Value'], \
                                                                                                    axis=1)
        df_month_hedge_error['Dynamic Hedge Error'] = df_month_hedge_error.apply(lambda x: x['Close'] - x['Dynamic Hedge Pfl Value'], axis=1)

        # The following piece is to filter data except last week as the long options expiry match weekly options
        last_week_start = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][-1]
        df_month_hedge_error["date_ind"] = df_month_hedge_error.apply(lambda x: 1 \
                                                                      if datetime.strptime(x['Hedge Error Date'], "%d-%b-%Y").date() <= last_week_start
                                                                      else 0, axis=1) 
        
        df_month_hedge_error = df_month_hedge_error[df_month_hedge_error["date_ind"] == 1]
        df_month_hedge_error = df_month_hedge_error[['Hedge Error Date', 'Expiry', 'iv', 'delta', 'delta_lag', 'Underlying Value', 'Strike Price', 'r', 'expiry_dt', 'Close', 'Dynamic Hedge Pfl Value', 'Dynamic Hedge Error']]
        df_month_hedge_error = df_month_hedge_error.rename(columns={"Hedge Error Date":"Date", "Expiry":"Month_Expiry"})
        mth_list_of_df.append(df_month_hedge_error)
        
    df_dynamic_hedge = pd.concat(mth_list_of_df, axis=0)

    df_dynamic_hedge.to_csv( output_path + "D1_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_dynamic_hedge.csv", index = False)

    return df_dynamic_hedge
  

In [25]:
#Compute Daily Dynamic Hedging Errors

###################################
# Common Variables initialisation #
###################################

#Find the monthly strike of option from mkt data to find the option to be hedged
#Every month, an option is hedged
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
dict_wkly_expiries_each_month = find_wkly_expries(mthly_expiries_list, holidays_list)

#Load all monthly mkt data
mthly_mkt_data = load_all_mthly_data(mthly_expiries_list, input_data_path, holidays_list, prod_type_lists=["FUT", "CE", "PE"], stock_ident=stock_ident)

#Load weekly risk free interest rates from futures
#This function to be changed based on updates on risk-free-interest 
dict_wkly_ir = generate_weekly_ir(dict_wkly_expiries_each_month, mthly_mkt_data, stock_ident)


df_dynamic_hedge = dynamic_hedge_pfl_error(mthly_expiries_list, dict_wkly_expiries_each_month, mthly_mkt_data,\
                                           prod_type, prod_moneyness, holidays_list, 
                                           output_data_path, stock_ident)
